In [2]:
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import numpy as np

data = pd.read_csv('train.csv')

In [3]:
x_train = data.drop(['year','AveragePrice','Total Bags'], axis = 1)
y_train = data['AveragePrice']

test = pd.read_csv('test.csv')
x_test = test.drop(['year','Total Bags'], axis = 1)

from sklearn.preprocessing import MinMaxScaler
m = MinMaxScaler()
df =data
m.fit_transform(df)

x_train01 = df.drop(['year','AveragePrice','Total Bags'], axis = 1)
y_train01 = df['AveragePrice']

In [4]:
x_train01.shape
split = int(0.75*x_train01.shape[0])
tr_X = x_train01[:split]
tr_Y = y_train01[:split]
te_x = x_train01[split:]
te_Y = y_train01[split:]

In [6]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score


clf = RandomForestRegressor(n_jobs = 4)        #Initialize the classifier object

parameters = {'n_estimators': [100,200,300,400,500,1000,2000]}    #Dictionary of parameters

scorer = make_scorer(r2_score)         #Initialize the scorer using make_scorer

grid_obj = GridSearchCV(clf,parameters,scoring=scorer)         #Initialize a GridSearchCV object with above parameters,scorer and classifier

grid_fit = grid_obj.fit(x_train01, y_train01)        #Fit the gridsearch object with X_train,y_train

best_clf = grid_fit.best_estimator_         #Get the best estimator. For this, check documentation of GridSearchCV object

unoptimized_predictions = (clf.fit(x_train01, y_train01)).predict(x_test)      #Using the unoptimized classifiers, generate predictions
optimized_predictions = best_clf.predict(x_test)        #Same, but use the best estimator

acc_unop = r2_score(te_Y, unoptimized_predictions)*100       #Calculate accuracy for unoptimized model
acc_op = r2_score(te_Y, optimized_predictions)*100         #Calculate accuracy for optimized model

print("Accuracy score on unoptimized model:{}".format(acc_unop))
print("Accuracy score on optimized model:{}".format(acc_op))

KeyboardInterrupt: 

In [6]:
df_soln = pd.DataFrame(data = [test['id'], optimized_predictions], index = None)
df = df_soln.transpose()
df['id'] = df['id'].astype(int)
df.columns = ['id','AveragePrice']
df.to_csv('soln opt pred.csv',index = False)